In [1]:
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt

In [2]:
with open(f"../results/training_in_forces.npy", "rb") as f:
    training_in = np.load(f)

with open(f"../results/training_out_forces.npy", "rb") as f:
    training_out = np.load(f)

In [4]:
training_in.shape

trainX = torch.tensor(training_in, dtype=torch.float32)
trainY = torch.tensor(training_out[:, -1, :], dtype=torch.float32)

In [5]:
class LSTMModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, layer_dim, output_dim):
        super(LSTMModel, self).__init__()
        self.hidden_dim = hidden_dim
        self.layer_dim = layer_dim
        self.lstm = nn.LSTM(input_dim, hidden_dim, layer_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x, h0=None, c0=None):
        # If hidden and cell states are not provided, initialize them as zeros
        if h0 is None or c0 is None:
            h0 = torch.zeros(self.layer_dim, x.size(0), self.hidden_dim).to(x.device)
            c0 = torch.zeros(self.layer_dim, x.size(0), self.hidden_dim).to(x.device)
        
        # Forward pass through LSTM
        out, (hn, cn) = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])  # Selecting the last output
        return out, hn, cn

In [ ]:
trainY.shape

torch.Size([6, 42])

In [6]:
# Initialize model, loss, and optimizer
model = LSTMModel(input_dim=42, hidden_dim=100, layer_dim=1, output_dim=42)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

# Training loop
num_epochs = 1000
h0, c0 = None, None  # Initialize hidden and cell states

for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()

    # Forward pass
    outputs, h0, c0 = model(trainX, h0, c0)

    # Compute loss
    loss = criterion(outputs, trainY)
    loss.backward()
    optimizer.step()

    # Detach hidden and cell states to prevent backpropagation through the entire sequence
    h0 = h0.detach()
    c0 = c0.detach()

    if (epoch+1) % 100 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

Epoch [100/1000], Loss: 121000528.0000


KeyboardInterrupt: 

In [7]:
trainX[-2:-1, :400, :].shape

torch.Size([1, 400, 42])

In [27]:
predicted_400, h400, c400 = model(trainX[-1:, :400, :], None, None)

In [28]:
predicted_401, _, _ = model(trainX[-1:, 400:401, :], h400, c400)

In [29]:
predicted_401_comp, _, _ = model(trainX[-1:, :401, :], None, None)

In [38]:
predict_test = training_in[-1, 400, :]
predicted_test, _, _ = model(torch.tensor(predict_test[None, None, :], dtype=torch.float32), h400, c400)

In [39]:
predicted_400

tensor([[ 6.3041e-02, -1.7005e+02,  6.7185e-02, -1.6943e+02,  1.1316e-01,
         -1.6972e+02,  1.0122e-01, -1.6944e+02, -2.9860e-02, -1.7122e+02,
         -1.0494e-01, -1.6915e+02, -1.5843e-01, -1.7043e+02, -1.3934e-01,
         -1.6981e+02, -1.1565e-01, -1.6958e+02, -8.8278e-02, -1.6972e+02,
         -5.0606e-02, -1.7051e+02, -4.0793e-02, -1.7044e+02, -4.6974e-02,
         -1.7028e+02, -5.1964e-02, -1.7052e+02, -4.6195e-02, -1.6997e+02,
         -5.1488e-02, -1.6970e+02, -3.3158e-02, -1.6926e+02,  2.4980e-02,
         -1.6986e+02,  7.2152e-02, -1.7043e+02,  8.1653e-02, -1.7038e+02,
          8.4323e-02, -1.6942e+02]], grad_fn=<AddmmBackward0>)

In [40]:
predicted_401

tensor([[ 6.3040e-02, -1.7005e+02,  6.7186e-02, -1.6943e+02,  1.1316e-01,
         -1.6972e+02,  1.0122e-01, -1.6944e+02, -2.9859e-02, -1.7122e+02,
         -1.0494e-01, -1.6915e+02, -1.5843e-01, -1.7043e+02, -1.3934e-01,
         -1.6981e+02, -1.1565e-01, -1.6958e+02, -8.8278e-02, -1.6972e+02,
         -5.0607e-02, -1.7051e+02, -4.0796e-02, -1.7044e+02, -4.6976e-02,
         -1.7028e+02, -5.1965e-02, -1.7052e+02, -4.6196e-02, -1.6997e+02,
         -5.1488e-02, -1.6970e+02, -3.3158e-02, -1.6927e+02,  2.4981e-02,
         -1.6986e+02,  7.2154e-02, -1.7043e+02,  8.1653e-02, -1.7038e+02,
          8.4325e-02, -1.6942e+02]], grad_fn=<AddmmBackward0>)

In [41]:
predicted_401_comp

tensor([[ 6.3040e-02, -1.7005e+02,  6.7186e-02, -1.6943e+02,  1.1316e-01,
         -1.6972e+02,  1.0122e-01, -1.6944e+02, -2.9859e-02, -1.7122e+02,
         -1.0494e-01, -1.6915e+02, -1.5843e-01, -1.7043e+02, -1.3934e-01,
         -1.6981e+02, -1.1565e-01, -1.6958e+02, -8.8278e-02, -1.6972e+02,
         -5.0607e-02, -1.7051e+02, -4.0796e-02, -1.7044e+02, -4.6976e-02,
         -1.7028e+02, -5.1965e-02, -1.7052e+02, -4.6196e-02, -1.6997e+02,
         -5.1488e-02, -1.6970e+02, -3.3158e-02, -1.6927e+02,  2.4981e-02,
         -1.6986e+02,  7.2154e-02, -1.7043e+02,  8.1653e-02, -1.7038e+02,
          8.4325e-02, -1.6942e+02]], grad_fn=<AddmmBackward0>)

In [42]:
predicted_test

tensor([[ 6.3040e-02, -1.7005e+02,  6.7186e-02, -1.6943e+02,  1.1316e-01,
         -1.6972e+02,  1.0122e-01, -1.6944e+02, -2.9859e-02, -1.7122e+02,
         -1.0494e-01, -1.6915e+02, -1.5843e-01, -1.7043e+02, -1.3934e-01,
         -1.6981e+02, -1.1565e-01, -1.6958e+02, -8.8278e-02, -1.6972e+02,
         -5.0607e-02, -1.7051e+02, -4.0796e-02, -1.7044e+02, -4.6976e-02,
         -1.7028e+02, -5.1965e-02, -1.7052e+02, -4.6196e-02, -1.6997e+02,
         -5.1488e-02, -1.6970e+02, -3.3158e-02, -1.6927e+02,  2.4981e-02,
         -1.6986e+02,  7.2154e-02, -1.7043e+02,  8.1653e-02, -1.7038e+02,
          8.4325e-02, -1.6942e+02]], grad_fn=<AddmmBackward0>)